# Predict dataset

In [1]:
import os
import csv
import cv2

from tensorflow import keras
from PIL import Image
from PIL.Image import Resampling
import numpy as np

In [2]:
dataset_folder = 'data_set/'
model_path = 'diagrams.h5'
result_csv_path = 'csv/results.csv'

batch_size = 1000

In [3]:
def batch_listdir(path:str, size=1):
    i = 0
    batch_list = []
    for element in os.scandir(path):
        if i < size:
            i += 1
            batch_list.append(element)
        else:
            yield batch_list
            i = 0
            batch_list = []

In [4]:
loaded_model = keras.models.load_model(model_path)

2022-07-15 04:35:18.047416: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-15 04:35:18.047760: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro


In [5]:
def evaluate_model(dataset_folder_path, model, writer, batch_size = 1):
    """Loads a dataset of images
        - dataset_folder_path is the path of the folder that contains the images
        - csv_path is the path of the CSV file that contains the labels of the images
        Returns: X_data, y_labeled
        - X_data is a numpy.ndarray containing the pixel data of an image X
        - y_labeled is a numpy.ndarray containing an int, the label Y for the image X in that index
    """
    n = 0
    for image_batch in batch_listdir(dataset_folder_path, batch_size):
        X_data = []
        not_loaded = []
        print(f'\rReading batch #{n}, size: {len(batch_size)}', flush=True, end=' ' * 50)
        for i in range(len(image_batch)):
            image = get_normalize_image(image_batch[i].path)
            if image is None:
                not_loaded.append(i)
            else:
                X_data.append(image)

        for i in not_loaded:
            image_batch.pop(i)
        X_data = np.array(X_data, dtype=np.uint8)
        X_data = keras.applications.densenet.preprocess_input(X_data)
        print(f'\rProcessing batch #{n}, size: {len(X_data)}', flush=True, end=' ' * 50)
        prob = model.predict_on_batch(X_data)
        bests = prob.argmax(axis=-1)
        prob_text = [['{:.6f}'.format(v) for v in p] for p in prob]
        rows = [[image_batch[i].name, str(bests[i])] + prob_text[i] for i in range(len(image_batch))]

        writer.writerows(rows)
        n += 1

In [6]:
def get_normalize_image(path: str):
    try:
        img = Image.open(path)
        img = img.convert('RGB').resize((224, 224), Resampling.LANCZOS)
        pixels = np.array(img, dtype=np.uint8)
        pixels_conv = cv2.cvtColor(pixels, cv2.COLOR_RGB2BGR)

        return pixels_conv
    except Exception as e:
        pass

In [7]:
csv_file = open(result_csv_path, mode='w')
csv_writer = csv.writer(csv_file)

In [8]:
evaluate_model(dataset_folder, loaded_model, csv_writer, batch_size)
csv_file.close()

Reading batch #0, size: 0                                                  

/Users/SergioAndresRodriguezTorres/Documents/escuelaing/Maestria/tesis/ML/venv/lib/python3.8/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Processing batch #0, size: 1000                                                  

2022-07-15 04:35:33.615464: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-15 04:35:33.616177: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Processing batch #3, size: 999                                                   

...

Reading batch #2208, size: 0                                                        

/Users/SergioAndresRodriguezTorres/Documents/escuelaing/Maestria/tesis/ML/venv/lib/python3.8/site-packages/PIL/Image.py:3011: DecompressionBombWarning: Image size (95486400 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Reading batch #2437, size: 0                                                        

/Users/SergioAndresRodriguezTorres/Documents/escuelaing/Maestria/tesis/ML/venv/lib/python3.8/site-packages/PIL/Image.py:3011: DecompressionBombWarning: Image size (134640000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Processing batch #2465, size: 1000                                                  